# Clean Stripe and identify plan duration using CM data

###Specifications

####  Stripe:

o Stripe- Add a new column to Stripe data and then perform the check with CM Stripe data to identify each item by Mon or Annual. There are few non-revenue Stripe data like fees /disputes. Move these to the bottom before doing this.


### Script
---

#### Imports,prepartions and functions

In [1]:
import pandas as pd
import os
!pip install XlsxWriter
import xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 2.1 MB/s 


###### Preparations
---


In [2]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')

#Set file paths
path = "/content/gdrive/My Drive/Data Science 2022/CM Audit/"

filestripe = path + "Stripe Data/Stripe Raw Data - 2022.09.xlsx"
filecm  =path + "Monthly Output/2022.09/CM_2022.09.xlsx"

outfile = path + "Monthly Output/2022.09/Stripe_2022.09.xlsx"


Mounted at /content/gdrive


#### Cleanup Stripe and  Compare with CM data to identify if the charges are annual or monthly

In [50]:
lstType = ['dispute','dispute_reversal','fee']

#Load file
if filestripe[-3:] == "csv": 
  month = filestripe[-9:-4]
  df = pd.read_csv(filestripe , na_filter=False, index_col=False)
else:
  month = filestripe[-10:-5]
  df = pd.read_excel(filestripe , na_filter=False, index_col=False)

############ Segregate Non-Revenue transactions based on 'Type'
#Move the non revenue 'reporting_category' to seperate sheet - Non revenue

dfmove = df[df['reporting_category'].isin(lstType)] #dataset to be moved to 'others' sheet
dfmove.insert(dfmove.shape[1],"Plan Type","")
dfclean = df[~df['reporting_category'].isin(lstType)] #clean dataset

dfs = dfclean.copy()

dfcm = pd.read_excel(filecm,sheet_name="Stripe" , na_filter=False, index_col=False)

dfmon = pd.merge(dfs, dfcm, right_on="Customer External ID", left_on="customer_id", how='left')

#Keep only stripe data along with plan
cmcols = dfcm.columns.tolist()
cmcols.remove('Plan Type')

mcols = dfmon.columns.tolist()
newcols = [x for x  in mcols if x not in cmcols  ]
dfnews = dfmon[newcols]

#Delete duplicate rows
dfnews.drop_duplicates(inplace=True)
print(dfnews.shape)
#Look for duplicate ChargeIDs. 
lstduplicate = dfnews[dfnews.duplicated('source_id')].source_id.to_list()
#For those chargeIDs find corresponding CM record with "Line Item Type" = 'subscription'. Get 'plan type' from that transaction
#Get all records from CM that has chargeid = dfduplicate['source_id']
dfcm_match = dfcm[dfcm['Charge ID'].isin(lstduplicate)]
dfcm_subs = dfcm_match[dfcm_match["Line Item Type"] == 'subscription']
#Iterate cm data to find plan type
lsthit=[]
for i,row in dfcm_subs.iterrows():
  for item in lstduplicate:
    if row['Charge ID'] == item:
      dfnews.loc[dfnews['source_id'] == item, ['Plan Type']] = row['Plan Type'] 
      lsthit.append(item)
#There are records with same customer id repeated twice. Sort out those
lstcus_duplicates =[ x for x in lstduplicate  if x not in lsthit  ]

#Remove those records from dfnews where chargeID is not in lsthit
for item in lstcus_duplicates:
  dfnews.drop(dfnews.index[(dfnews['source_id'] == item) & (dfnews['Plan Type'] == 'Yearly')],inplace=True)

dfnews.drop_duplicates(inplace=True)

#Sort by plan type
dfnews = dfnews.sort_values('Plan Type',ascending=False)

writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
dfnews.to_excel(writer,sheet_name="Cleaned Data", index=False)
dfmove.to_excel(writer,sheet_name="Non Revenue records", index=False)

writer.save()
writer.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_

(31680, 65)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [48]:
print(df.shape)
print(dfmove.shape)
print(dfnews.shape)

(31781, 64)
(237, 65)
(31544, 65)


In [49]:
31544+237

31781

In [45]:
for item in lstcus_duplicates:

  dfnews.drop(dfnews.index[(dfnews['source_id'] == item) & (dfnews['Plan Type'] == 'Yearly')],inplace=True)
  #print(dfnews.index[(dfnews['source_id'] == item) & (dfnews['Plan Type'] == 'Yearly') ])
  print(dfnews.shape)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(31559, 65)
(31558, 65)
(31557, 65)
(31556, 65)
(31555, 65)
(31554, 65)
(31553, 65)
(31552, 65)
(31551, 65)
(31550, 65)
(31549, 65)
(31548, 65)
(31547, 65)
(31546, 65)
(31545, 65)
(31544, 65)


In [29]:
print(len(lstcus_duplicates))
print(len(lstduplicate))
print(len(lsthit))

16
136
120


In [17]:
lsthit=[]
for i,row in dfcm_subs.iterrows():
  for item in lstduplicate:
    if row['Charge ID'] == item:
      dfnews.loc[dfnews['source_id'] == item, ['Plan Type']] = row['Plan Type'] 
      lsthit.append(item)
#There are records with same customer id repeated twice. Sort out those
lstcus_duplicates =[ x for x in lstduplicate  if x not in lsthit  ]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [22]:

for i,row in dfcm_match.iterrows():
  for item in lstcus_duplicates:
    if row['Charge ID'] == item:
      dfnews.loc[dfnews['source_id'] == item, ['Plan Type']] = row['Plan Type'] 
      print(row['Line Item Type'])
    

prorate
prorate
refund
refund
prorate
prorate
prorate
prorate
refund
refund
refund
refund
refund
refund
prorate
prorate
refund
refund
refund
refund


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [20]:
lstcus_duplicates

['re_2LdKWmEDdwUf7nb70XFKioDv',
 'ch_2Led9zEDdwUf7nb71Is5upFE',
 're_2Lg3hAEDdwUf7nb70zOWLfWa',
 're_2LgVveEDdwUf7nb716DhXYN7',
 're_2LgVsqEDdwUf7nb715IMN7aL',
 're_2LgukyEDdwUf7nb70pRDmxBU',
 're_2LhIUGEDdwUf7nb71FogUjXD',
 're_2LhCMoEDdwUf7nb70dqu9qhy',
 're_2LhVJdEDdwUf7nb70H7leFVw',
 're_2LhPW2EDdwUf7nb70ZtoLJ0d',
 'ch_2Lj7DnEDdwUf7nb71XLeTDQ9',
 'ch_2Lk2B8EDdwUf7nb71drh8Fi6',
 're_2LkIfaEDdwUf7nb70rXZfSUr',
 're_2Lk2B8EDdwUf7nb71zbkrOG2',
 're_2Lke34EDdwUf7nb70pJGafme',
 'ch_2LnUYBEDdwUf7nb7086LDabF']

In [19]:
dfnews.shape

(31560, 65)

In [46]:
31544 + 237

31781

In [15]:
31797-16

31781

In [10]:
df.shape

(31781, 64)

In [11]:
dfmove.shape

(237, 65)

In [7]:
dfcm_match.to_excel(path + 'test.xlsx')

In [6]:
dfcm_subs = dfcm_match[dfcm_match["Line Item Type"] == 'subscription']
dfcm_subs.shape

(120, 20)

In [16]:
lstduplicate = dfnews[dfnews.duplicated('source_id')].source_id.to_list()


In [22]:
dfcm_match = dfcm[dfcm['Charge ID'].isin(lstduplicate)]

In [23]:
dfcm_match.shape

(221, 20)

In [18]:
dfcm.columns

Index(['Customer Name', 'Customer ID', 'Customer UUID', 'Customer External ID',
       'Charge ID', 'Invoice ID', 'Charge Timestamp UTC', 'Charge Timestamp',
       'Invoice Timestamp UTC', 'Invoice Timestamp', 'Payment / Refund',
       'Invoice Status', 'Line Item ID', 'Line Item Name', 'Line Item Type',
       'Line Item Value', 'Line Item Currency',
       'Line Item Value Account Currency', 'Account Currency', 'Plan Type'],
      dtype='object')

In [10]:
dfduplicate['source_id']

96       ch_2Ld3oTEDdwUf7nb718V4weoD
568      ch_2LdDqEEDdwUf7nb718mJjcSC
574      ch_2LdDypEDdwUf7nb70xNCBFCl
660      ch_2LdFGSEDdwUf7nb70hpjwaMW
1036     ch_2LdKWmEDdwUf7nb70XWaYPvN
                    ...             
37294    ch_2LnMGdEDdwUf7nb71fyc5pVg
37939    ch_2LnT0qEDdwUf7nb709mc5wk6
38053    ch_2LnUYBEDdwUf7nb7086LDabF
39431    ch_2LnlhlEDdwUf7nb71h9u2PDJ
39897    ch_2Lnq1vEDdwUf7nb70Lb9gnG7
Name: source_id, Length: 136, dtype: object

In [11]:
dfduplicate['source_id'].unique

<bound method Series.unique of 96       ch_2Ld3oTEDdwUf7nb718V4weoD
568      ch_2LdDqEEDdwUf7nb718mJjcSC
574      ch_2LdDypEDdwUf7nb70xNCBFCl
660      ch_2LdFGSEDdwUf7nb70hpjwaMW
1036     ch_2LdKWmEDdwUf7nb70XWaYPvN
                    ...             
37294    ch_2LnMGdEDdwUf7nb71fyc5pVg
37939    ch_2LnT0qEDdwUf7nb709mc5wk6
38053    ch_2LnUYBEDdwUf7nb7086LDabF
39431    ch_2LnlhlEDdwUf7nb71h9u2PDJ
39897    ch_2Lnq1vEDdwUf7nb70Lb9gnG7
Name: source_id, Length: 136, dtype: object>